In [ ]:
import json
import pandas as pd
import numpy as np
import re

In [ ]:
file_dir = 'C:/Users/Local User/Documents/GitHub/Movies-ETL/'

In [ ]:
with open(f'{file_dir}wikipedia-movies.json', mode='r') as file:
    wiki_movies_raw = json.load(file)
len(wiki_movies_raw)

In [ ]:
# First 5 records
wiki_movies_raw[:5]

In [ ]:
# Last 5 records
wiki_movies_raw[-5:]

In [ ]:
# Some records in the middle
wiki_movies_raw[3600:3605]

In [ ]:
kaffle_metadata = pd.read_csv(f'{file_dir}movies_metadata.csv', low_memory=False)
ratings = pd.read_csv(f'{file_dir}ratings.csv')

In [ ]:
wiki_movies_df = pd.DataFrame(wiki_movies_raw)
wiki_movies_df.head()

In [ ]:
wiki_movies = [movie for movie in wiki_movies_raw if ('Director' in movie or 'Directed by' in movie) and 'imdb_link' in movie and 'No. of episodes' not in movie]
len(wiki_movies)

In [ ]:
wiki_movies_df = pd.DataFrame(wiki_movies)
wiki_movies_df

In [ ]:
wiki_movies_df[wiki_movies_df['Arabic'].notnull()]

In [ ]:
wiki_movies_df[wiki_movies_df['Arabic'].notnull()]['url']

In [ ]:
wiki_movies_df.columns.tolist()

In [ ]:
wiki_movies_df.value_counts('McCune–Reischauer')


In [ ]:
def clean_movie(movie):
    movie = dict(movie) # create a non-distructive copy
    alt_titles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles
    
    # merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Directed By', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release Date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production Company(s)')
    change_column_name('Productioncompany ', 'Production Company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release date', 'Release Date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')
    change_column_name('Production company(s)', 'Production Company(s)')
    
    return movie

In [ ]:
clean_movies = [clean_movie(movie) for movie in wiki_movies]

In [ ]:
wiki_movies_df = pd.DataFrame(clean_movies)
sorted(wiki_movies_df.columns.tolist())

In [ ]:
wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
print(len(wiki_movies_df))
wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
print(len(wiki_movies_df))
wiki_movies_df.head()

In [ ]:
[[column,wiki_movies_df[column].isnull().sum()] for column in wiki_movies_df.columns]

In [ ]:
# 8.3.7 
[column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]

In [ ]:
wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) *0.9]
wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]
wiki_movies_df.head()

In [ ]:
wiki_movies_df.dtypes

In [ ]:
box_office = wiki_movies_df['Box office'].dropna()
box_office

In [ ]:
def is_not_a_string(x):
    return type(x) != str
box_office[box_office.map(is_not_a_string)]

In [ ]:
box_office[box_office.map(lambda x: type(x) != str)]

In [ ]:
box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)

In [ ]:
form_one = r'\$\d+\.?\d*\s*[mb]illion'

In [ ]:
box_office.str.contains(form_one, flags=re.IGNORECASE, na=False).sum()

In [ ]:
form_two = r'\$\d{1,3}(?:,\d{3})+'
box_office.str.contains(form_two, flags=re.IGNORECASE, na=False).sum()

In [ ]:
matches_form_one = box_office.str.contains(form_one, flags=re.IGNORECASE, na=False)
matches_form_two = box_office.str.contains(form_two, flags=re.IGNORECASE, na=False)

In [ ]:
box_office[~matches_form_one & ~matches_form_two].tolist()

In [ ]:
form_one = matches_form_one = box_office.str.contains(form_one, flags=re.IGNORECASE, na=False)
matches_form_two = box_office.str.contains(form_two, flags=re.IGNORECASE, na=False)
form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+($!\s[mb]illi?on)'

In [ ]:
box_office = box_office.str.replace(r'\$.*[---](?![a-z])', '$', regex=True)

In [ ]:
box_office.str.extract(f'({form_one}|{form_two})')

In [ ]:
def parse_dollars(s):
    # if x is not a string, return NaN
    if type(s) != str:
        return np.nan
    
    # if input is of the form $###.# million
    if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):
        
        # remove dollar sign and " million"
        s = re.sub('\$|\s|[a-zA-Z]','', s)
        
        # convert to float and mulitply by a million
        value = float(s) * 10**6
        
        # return value
        return value
        
    # if input is of the form $###.# billion
    elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):
        ## remove dollar sign and " billion"
        s = re.sub('\$|\s|[a-zA-Z]', '', s)
        
        # convert to float and multiply by a billion
        value = float(s) * 10**9
        
        # return value
        return value
    
    # if input is of the form $###,###,###
    elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):
        
        # remove dollar sign and commas
        s = re.sub('\$|, ', '', s)
        # convert to float
        value = float(s)
        # return value
        return value
    
    # otherwise, return NaN
    else:
        return np.nan

In [ ]:
wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
wiki_movies_df['box_office']

In [ ]:
wiki_movies_df.columns.tolist()

In [ ]:
wiki_movies_df.drop('Box office', axis=1, inplace=True)

In [ ]:
budget = wiki_movies_df['Budget'].dropna()

In [ ]:
# convert any listings to strings
budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)

In [ ]:
# remove any values between a dollar sign and a hyphen for budgets given in ranges
budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)

In [ ]:
# Use same pattern matches created previously and apply without modifications to the budget data
matches_form_one = budget.str.contains(form_one, flags=re.IGNORECASE, na=False)
matches_form_two = budget.str.contains(form_two, flags=re.IGNORECASE, na=False)
budget[~matches_form_one & ~matches_form_two]